# Band Pass Filter 테스트
- Octave Band scale의 Band Pass Filter 생성
- Butterworth 필터
- FIR Filter 처리 가능여부


In [1]:
'''
# 라이브러리 정의
 - 시스템 라이브러리
 - 오디오 관련 라이브러리
 - 연산 및 Plot 관련 라이브러리 
 - 샤용자 라이브러리
'''

# Import Systems 
import struct
import io
import os
import sys
import time
import math
import platform

# Import Audio
import wave
import pyaudio
import librosa

import numpy as np
import scipy
import scipy.signal as sig
import matplotlib.pyplot as plt
import soundfile

# User Libraries
import pyOssWavfile
import pyRoomAcoustic as room
import pyOssDebug as dbg
import pyOssFilter



In [2]:
# .wav File Open

# Path of audiofiles folder
audio_dir = os.path.join(os.getcwd(), 'audiofiles')

# Define wave file name text to open
# Path + wave file name

#audio_fname = "TyndallBruceMonument.wav"
#audio_fname = "ElvedenHallMarbleHall.wav'"
#audio_fname = "EmptyApartmentBedroom.wav"
#audio_fname = "FalklandPalaceRoyalTennisCourt.wav"
#audio_fname = "InsidePiano.wav"
audio_fname = "MaesHowe.wav"
#audio_fname = "SportsCentreUniversityOfYork.wav"
#audio_fname = "StairwayUniversityOfYork.wav"
#audio_fname = "StAndrewsChurch.wav"

wav_fname = os.path.join(audio_dir, audio_fname)

ossWav_fmt_chunk = pyOssWavfile.read_format(wav_fname)
ossWav_struct_fmt = pyOssWavfile.extractWavFmtChunk(ossWav_fmt_chunk)

if ossWav_struct_fmt.bitdepth == 24:        # Bitdepth가 24bit일 때는 pysoundfile을 이용하여 wav data를 불러옴
    ossWav_data, ossWav_struct_fmt.fs = soundfile.read(wav_fname)
else:                                       # 그 외에는 scipy.io.wavfile을 수정한 함수 pyOssWavfile을 이용하여 wav data를 불러옴
    ossWav_fmt_chunk, ossWav_data, ossWav_length = pyOssWavfile.read(wav_fname)

ossWav_time = ossWav_data.shape[0] / ossWav_struct_fmt.fs


In [4]:
p = pyaudio.PyAudio()


IDX_ASIO = pyaudio.paASIO
IDX_COREAUDIO = pyaudio.paCoreAudio     # = 5

if 'darwin' in platform.system():
    IDX = pyaudio.paCoreAudio
elif 'linux' in platform.system():
    IDX = pyaudio.paALSA
else:
    IDX = pyaudio.paWASAPI 


for i in range(p.get_device_count()):
    device_index = p.get_device_info_by_index(i)
    print(device_index)
    # if ('ASIO' in device_index):
    if ('Steinberg UR44' in device_index):
        break

print(device_index)

for i in range(p.get_host_api_count()):
    api_index = p.get_host_api_info_by_index(i)
    print(api_index)


api_index_key = list(api_index.keys())
api_index_value = list(api_index.values())



{'index': 4, 'structVersion': 2, 'name': 'ZoomAudioDevice', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.01, 'defaultLowOutputLatency': 0.03333333333333333, 'defaultHighInputLatency': 0.1, 'defaultHighOutputLatency': 0.042666666666666665, 'defaultSampleRate': 48000.0}
{'index': 0, 'structVersion': 1, 'type': 5, 'name': 'Core Audio', 'deviceCount': 5, 'defaultInputDevice': 1, 'defaultOutputDevice': 3}


In [ ]:
p = pyaudio.PyAudio()

stream = p.open(
    format = pyaudio.paFloat32,
    channels = 1,
    rate = RATE,
    input = True,
    output = False,
    frames_per_buffer = BUFFER
)

fig = plt.figure()
line1 = plt.plot([],[])[0]
line2 = plt.plot([],[])[0]

r = range(0,int(RATE/2+1),int(RATE/BUFFER))
l = len(r)